In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
interaction=pd.read_csv("../data/ratings.csv")
tag_interaction=pd.read_csv("../data/tags.csv")

In [3]:
tag_interaction.dropna(inplace=True)

In [4]:
user_movie_tag_data=pd.DataFrame(tag_interaction.groupby(["userId","movieId"])["tag"].apply(lambda x: "|".join(x)))
user_movie_tag_data

tag
userId movieId                                    
3      260                          classic|sci-fi
4      1732             dark comedy|great dialogue
       7569                       so bad it's good
       44665                  unreliable narrators
       115569                                tense
...                                            ...
162521 8874                       cornetto triolgy
       51255                      cornetto trilogy
       66934    Nathan Fillion|Neil Patrick Harris
       103341                     cornetto trilogy
162534 189169              comedy|disabled|robbery

[305341 rows x 1 columns]

In [5]:
user_movie_tag_with_ratings=user_movie_tag_data.merge(interaction,on=["userId","movieId"],how="left")

In [6]:
user_metadata=user_movie_tag_with_ratings[["userId","tag"]]

In [7]:
user_metadata=pd.DataFrame(user_metadata.groupby("userId")["tag"].apply(lambda x: "|".join(x)))

In [8]:
user_metadata.isnull().sum()

tag    0
dtype: int64

In [9]:
user_metadata

,tag
userId,
3,classic|sci-fi
4,dark comedy|great dialogue|so bad it's good|un...
19,Oscar (Best Supporting Actress)|adventure|anim...
20,bah
43,Clint Eastwood
...,...
162495,Nudity (Full Frontal - Notable)
162501,crazy wife|crime|the wife did it
162512,the book was better|the book was better


In [10]:
user_metadata.rename(columns={"tag":"user_tags"},inplace=True)

In [11]:
user_metadata = pd.DataFrame(user_metadata["user_tags"].apply(lambda x: "|".join(set(x.split("|")))))
user_metadata

,user_tags
userId,
3,classic|sci-fi
4,unreliable narrators|unpredictable|so bad it's...
19,post-apocalyptic|fantasy|anime|adventure|Hayao...
20,bah
43,Clint Eastwood
...,...
162495,Nudity (Full Frontal - Notable)
162501,crime|the wife did it|crazy wife
162512,the book was better


In [12]:
user_metadata

,user_tags
userId,
3,classic|sci-fi
4,unreliable narrators|unpredictable|so bad it's...
19,post-apocalyptic|fantasy|anime|adventure|Hayao...
20,bah
43,Clint Eastwood
...,...
162495,Nudity (Full Frontal - Notable)
162501,crime|the wife did it|crazy wife
162512,the book was better


In [13]:
most_relevant_tags = pd.read_csv("../data/most_relevant_tags.csv")

In [14]:
cold_start_tags = most_relevant_tags["tag"].tolist()

In [15]:
cold_start_tags = "|".join(cold_start_tags)

In [16]:
cold_start_tags

'original|mentor|catastrophe|great ending|storytelling'

In [17]:
def create_cold_start_tags(user_tags,cold_start_tags):
    if len(user_tags.split("|"))<=2:
        return user_tags + "|" + cold_start_tags
    return user_tags

In [18]:
create_cold_start_tags("sci-fi|classic|horror",cold_start_tags)

'sci-fi|classic|horror'

In [19]:
user_metadata["user_tags"] = user_metadata["user_tags"].apply(lambda x: create_cold_start_tags(x,cold_start_tags))

In [20]:
user_metadata["user_tag_length"] = user_metadata["user_tags"].apply(lambda x:len(x.split("|")))
user_metadata["user_tag_length"].describe()

count    14592.000000
mean        38.222519
std        234.937633
min          3.000000
25%          6.000000
50%          7.000000
75%         16.000000
max      20765.000000
Name: user_tag_length, dtype: float64

In [21]:
# user_metadata = user_metadata["user_tag_length"].apply(lambda x: )

In [22]:
user_metadata = user_metadata[(user_metadata["user_tag_length"]<=40) & (user_metadata["user_tag_length"]>=2)]

In [23]:
user_metadata

,user_tags,user_tag_length
userId,,
3,classic|sci-fi|original|mentor|catastrophe|gre...,7
4,unreliable narrators|unpredictable|so bad it's...,11
19,post-apocalyptic|fantasy|anime|adventure|Hayao...,8
20,bah|original|mentor|catastrophe|great ending|s...,6
43,Clint Eastwood|original|mentor|catastrophe|gre...,6
...,...,...
162495,Nudity (Full Frontal - Notable)|original|mento...,6
162501,crime|the wife did it|crazy wife,3
162512,the book was better|original|mentor|catastroph...,6


In [24]:
user_metadata.to_csv("../data/user_metadata.csv")